#### Problem description:
- When a vaccine for COVID-19 is ready, possibly it will not be available immediately in enough quantity to immunize everyone in every region. 

#### Objective:
- In a prescriptive analytic approach, this work proposes a linear programming model aimed to optimize a city vaccination campaign in a way to minimize the number of possible deaths in a scenario where just part of the vaccines arrives in certain regions. Here we are simulating such a scenario using São Paulo city hall data.

#### Outcome:
- A dashboard was created (link below) using the results generated here.
- https://dataplatform.cloud.ibm.com/dashboards/7e0a7415-0514-4a82-838a-2f9e8949ff05/view/0768fc2738a832ff6dd7e6e4079d2a547937775bb2bb870280d07b490f317997f36d1a92c87a4f088c150d61faba475a9f

#### References:
- https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/saude/COVID19_Relatorio_SItuacional_SMS_20200529.pdf
- https://www.prefeitura.sp.gov.br/cidade/secretarias/subprefeituras/subprefeituras/dados_demograficos/index.php?p=12758
- https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm?s_cid=mm6912e2_w#F1_down

In [41]:
#Imports

import ibm_boto3
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
from docplex.mp.model import Model
from ibm_botocore.client import Config
from docplex.mp.environment import Environment
from sklearn.linear_model import LinearRegression

In [42]:
#Project's bucket 

ibm_api_key_id = '***'
ibm_auth_endpoint = 'https://iam.ng.bluemix.net/oidc/token'
endpoint_url = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

res = ibm_boto3.resource(service_name='s3',
    ibm_api_key_id = ibm_api_key_id,
    ibm_auth_endpoint = ibm_auth_endpoint,
    config=Config(signature_version = 'oauth'),
    endpoint_url = endpoint_url)

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id = ibm_api_key_id,
    ibm_auth_endpoint = ibm_auth_endpoint,
    config = Config(signature_version = 'oauth'),
    endpoint_url = endpoint_url)

- Loading the São Paulo data for its districts, populations, and number of cases ...

In [43]:
obj = cos.get_object(Bucket=bucket_in_out, Key='SP_districts_and_number_of_cases_v2.xlsx')
df_districts = pd.read_excel(obj['Body'], sheet_name='districts')
df_districts.head()

,Distrito,População (2010)
0,Água Rasa,84963
1,Alto de Pinheiros,43117
2,Anhanguera,65859
3,Aricanduva,89622
4,Artur Alvim,105269


- Estimating the population in 2020 considering an increase of 10% relative to 2010 ...

In [44]:
df_districts["Population (2020)"] = df_districts["População (2010)"].apply(lambda x: int(x * 1.1))

- Adding cases per district and calculating the proportional incidence per district ...

In [45]:
obj = cos.get_object(Bucket=bucket_in_out, Key='SP_districts_and_number_of_cases_v2.xlsx')
df_number_of_cases = pd.read_excel(obj['Body'], sheet_name='number of cases until 0529')
df_districts.dropna(inplace=True)
df_districts = df_districts.merge(df_number_of_cases, how='outer', on='Distrito')
df_districts["Incidence"] = round(df_districts["Nb. of cases"] / df_districts["Population (2020)"],5)
df_districts.head()

,Distrito,População (2010),Population (2020),Nb. of cases,Incidence
0,Água Rasa,84963,93459,168,0.00180
1,Alto de Pinheiros,43117,47428,83,0.00175
2,Anhanguera,65859,72444,89,0.00123
3,Aricanduva,89622,98584,191,0.00194
4,Artur Alvim,105269,115795,207,0.00179


- Normally distributing each district population in three risk groups (low, medium, and high) ...

In [46]:
def distribute_pop_into_risk_groups(pop):
    distribution = truncnorm.rvs(a=-1, b=+1, size=pop)
    distribution = distribution.round().astype(int)
    return [len(np.where(distribution==-1)[0]), len(np.where(distribution==0)[0]), len(np.where(distribution==1)[0])]

distributions = df_districts["Population (2020)"].apply(distribute_pop_into_risk_groups)

flag_first = True
for distribution in distributions:
    if flag_first:
        distribution_low_risk = [distribution[0]]
        distribution_med_risk = [distribution[1]]
        distribution_hig_risk = [distribution[2]]
    else:
        distribution_low_risk.append(distribution[0])
        distribution_med_risk.append(distribution[1])
        distribution_hig_risk.append(distribution[2])
    flag_first=False

df_districts["Low Risk Pop."] = distribution_low_risk 
df_districts["Med. Risk Pop."] = distribution_med_risk
df_districts["High Risk Pop."] = distribution_hig_risk

df_districts.head()

,Distrito,População (2010),Population (2020),Nb. of cases,Incidence,Low Risk Pop.,Med. Risk Pop.,High Risk Pop.
0,Água Rasa,84963,93459,168,0.00180,20623,52403,20433
1,Alto de Pinheiros,43117,47428,83,0.00175,10586,26467,10375
2,Anhanguera,65859,72444,89,0.00123,15693,40881,15870
3,Aricanduva,89622,98584,191,0.00194,21719,55537,21328
4,Artur Alvim,105269,115795,207,0.00179,25420,64977,25398


- Setting the number of available vaccines to be distributed (simulation) ...

In [47]:
population = int(np.sum(df_districts["Population (2020)"]))
#nb_vaccines = int(np.sum(population) * .75)
nb_vaccines = 7.8*10**6
print("Population:", "{:,}".format(population))
print("Number of vaccines:", "{:,}".format(nb_vaccines))

Population: 12,378,835
Number of vaccines: 7,800,000.0


- Defining the mortality rate for each of the three risk groups ...

In [48]:
mortality_rate_per_risk_group = [0.001, 0.027, 0.158]
print("Mortality rate per risk group:", mortality_rate_per_risk_group)

Mortality rate per risk group: [0.001, 0.027, 0.158]


- Modeling and optimizing ...

In [49]:
mdl = Model(name="COVID-19 Vaccination Campaign")

#Variables

#The number of vaccines per district per risk group.

nb_vaccines_per_district_per_risk_group = list()
for i in range(df_districts.shape[0]):
    risk_group = list()
    for j in range(len(mortality_rate_per_risk_group)):
        varname = "nb_vaccines_per_district_per_risk_group_" + str(i) + "_" + str(j)
        risk_group.append(mdl.integer_var(name=varname))
    nb_vaccines_per_district_per_risk_group.append(risk_group)

#The total number of vaccines.
tot_vaccines = mdl.sum(nb_vaccines_per_district_per_risk_group[i][j] 
                    for j in range(len(mortality_rate_per_risk_group)) 
                    for i in range(df_districts.shape[0]))

#Variable to minimize.
deaths_vaccination_scenario = mdl.sum(((df_districts[["Low Risk Pop.","Med. Risk Pop.","High Risk Pop."]].iloc[i][j] 
                                        - nb_vaccines_per_district_per_risk_group[i][j])
                                        * mortality_rate_per_risk_group[j])
                                        * df_districts["Incidence"][i]
                                    for j in range(len(mortality_rate_per_risk_group))
                                    for i in range(df_districts.shape[0]))
                        

#Constraints
constr_nb = 0

#The number of vaccines per region per risk group can not be higher than the respective population.
for i in range(df_districts.shape[0]):
    for j in range(len(mortality_rate_per_risk_group)):
        constr_nb += 1
        mdl.add_constraints([nb_vaccines_per_district_per_risk_group[i][j] <= df_districts[["Low Risk Pop.","Med. Risk Pop.","High Risk Pop."]].iloc[i][j]],  "constr_nb_" + str(constr_nb))

#The total given vaccines can not be higher than the total available.
constr_nb += 1
mdl.add_constraint((tot_vaccines <= nb_vaccines), "constr_nb_" + str(constr_nb))

#Optimization.
mdl.minimize(deaths_vaccination_scenario)
mdl.solve()
mdl.print_information()
print()

#Summary

total_pop = df_districts["Population (2020)"].sum()

#deaths in a nonvaccination scenario
deaths_nonvaccination_scenario = np.sum([df_districts[["Low Risk Pop.","Med. Risk Pop.","High Risk Pop."]].iloc[i][j] 
                                         * mortality_rate_per_risk_group[j] 
                                         * df_districts["Incidence"][i]
                                    for j in range(len(mortality_rate_per_risk_group)) 
                                    for i in range(df_districts.shape[0])])
deaths_nonvaccination_scenario = int(round(deaths_nonvaccination_scenario))

#deaths (minimized) in a vaccination scenario
deaths_vaccination_scenario = int(round(deaths_vaccination_scenario.solution_value))

#Campaign efficiency in percentage
campaign_efficiency_pct = round((deaths_nonvaccination_scenario - deaths_vaccination_scenario) / deaths_nonvaccination_scenario * 100, 1)
                                       
#Dataframe with the vaccines' distribution - to be consumed in the dashboard
list_vaccines_per_district_per_risk_group = [[df_districts["Distrito"].values[i], j, 
                                              df_districts[["Low Risk Pop.","Med. Risk Pop.","High Risk Pop."]].iloc[i][j],
                                              int(nb_vaccines_per_district_per_risk_group[i][j].solution_value)] 
                                            for j in range(len(mortality_rate_per_risk_group))
                                            for i in range(df_districts.shape[0])]
df_vaccines_per_district_per_risk_group = pd.DataFrame(list_vaccines_per_district_per_risk_group, columns=["Distrito", "Group", "Population", "Number of Vaccines"])

#presentation
print("Population", "{:,}".format(total_pop))
print("Number of vaccines", "{:,}".format(tot_vaccines.solution_value))
print("Deaths nonvaccination scenario", "{:,}".format(deaths_nonvaccination_scenario))
print("Deaths vaccination scenario", "{:,}".format(deaths_vaccination_scenario))
print("Campaign efficiency %", "{:,}".format(campaign_efficiency_pct))

Model: COVID-19 Vaccination Campaign
 - number of variables: 288
   - binary=0, integer=288, continuous=0
 - number of constraints: 289
   - linear=289
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP

Population 12,378,835
Number of vaccines 7,800,000
Deaths nonvaccination scenario 1,048
Deaths vaccination scenario 70
Campaign efficiency % 93.3


- Saving the dataframe for the dashboard ...

In [50]:
# Creating and saving dataframe with the summary to be used in the dashboards.
summary = [total_pop, df_districts.shape[0], nb_vaccines, deaths_nonvaccination_scenario, deaths_vaccination_scenario, campaign_efficiency_pct]
column_names = ["Population", "Nb. of Districts", "Nb. of Vaccines", "Deaths Nonvacc. Scenario", "Deaths Vaccination Scenario", "Campaign Efficiency"]
df_challenge_covid19_summary = pd.DataFrame([summary], columns=column_names).rename_axis("summary")
csv_buffer = StringIO()
df_challenge_covid19_summary.to_csv(csv_buffer, index=True)
res.Object(bucket_in_out, 'df_challenge_covid19_summary.csv').put(Body=csv_buffer.getvalue())

#Joining dataframes and saving (to be used for the dashboard)
df_districts.set_index(["Distrito"], inplace=True)
df_risk_low = df_vaccines_per_district_per_risk_group.query("Group == 0")[["Distrito", "Number of Vaccines"]].rename(columns={'Number of Vaccines':'Nb. of Vaccines Low Risk Pop.'}).set_index("Distrito")
df_risk_med = df_vaccines_per_district_per_risk_group.query("Group == 1")[["Distrito", "Number of Vaccines"]].rename(columns={'Number of Vaccines':'Nb. of Vaccines Med. Risk Pop.'}).set_index("Distrito")
df_risk_hig = df_vaccines_per_district_per_risk_group.query("Group == 2")[["Distrito", "Number of Vaccines"]].rename(columns={'Number of Vaccines':'Nb. of Vaccines High Risk Pop.'}).set_index("Distrito")
df_districts = df_districts.join(df_risk_low, how='left')
df_districts = df_districts.join(df_risk_med, how='left')
df_districts = df_districts.join(df_risk_hig, how='left')
csv_buffer = StringIO()
df_districts.to_csv(csv_buffer, index=True)
res.Object(bucket_in_out, 'df_challenge_covid19_vaccines.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '0749a589-6570-434b-a9b4-92cff6913c7e',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Jul 2020 22:33:04 GMT',
   'x-clv-request-id': '0749a589-6570-434b-a9b4-92cff6913c7e',
   'server': 'Cleversafe/3.14.13.32',
   'x-clv-s3-version': '2.5',
   'x-amz-request-id': '0749a589-6570-434b-a9b4-92cff6913c7e',
   'etag': '"2b870e262b9ad5bbdea3145ed91be1f1"',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2b870e262b9ad5bbdea3145ed91be1f1"'}

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>